In [1]:
import time
import random
import math

In [2]:
pessoas = [("Amanda", "CWB"),
           ("Pedro", "GIG"),
           ("Marcos", "POA"),
           ("Priscila", "FLN"),
           ("Jessica", "CNF"),
           ("Paulo", "GYN"),]

In [3]:
destino = "GRU"